In [8]:
from utils import pickle_load

train_embs = pickle_load(r"c:\Users\msi\Downloads\train_embs.pkl")
test_embs = pickle_load(r"c:\Users\msi\Downloads\test_embs.pkl")

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [7]:
len(train_embs)

28466